In [6]:

import scanpy as sc
import pandas as pd
from pathlib import Path


In [10]:
!pip install scanpy
!pip install pathlib
!pip install pandas
!pip install synapseclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6


In [16]:
import pandas as pd
import os
synapse_ids_summary = pd.read_csv('/sbgenomics/workspace/data/synapse_ids_summary.csv')
synapse_ids_summary


import os
import synapseclient

# Log in to Synapse
syn = synapseclient.Synapse()
syn.login(email='email',authToken="token")


# Iterate through each patient's Synapse IDs and download files
for idx, row in synapse_ids_summary.iterrows():
    participant_id = row['HTAN_Participant_ID']
    # Create a directory for each participant
    os.makedirs(participant_id, exist_ok=True)

    # Download each file
    for file_type in ['Barcodes', 'Features', 'Matrix']:
        synapse_id = row[f'{file_type} Synapse ID']
        # Get the file entity to keep the original filename
        file_entity = syn.get(synapse_id, downloadLocation=participant_id)
        # Move the file to the participant directory if it's downloaded in a subfolder
        original_file_path = os.path.join(participant_id, file_entity.name)
        new_file_path = os.path.join(participant_id, os.path.basename(file_entity.name))
        os.rename(original_file_path, new_file_path)
        print(f"Downloaded {file_entity.name} for {participant_id}")

print("All files downloaded.")

Welcome, davidenoma!

Downloaded HT029B1-S1PC-barcodes.tsv.gz for HTA12_118
Downloaded HT029B1-S1PC-features.tsv.gz for HTA12_118
Downloaded HT029B1-S1PC-matrix.mtx.gz for HTA12_118
Downloaded HT035B1-S1PA-barcodes.tsv.gz for HTA12_119
Downloaded HT035B1-S1PA-features.tsv.gz for HTA12_119
Downloaded HT035B1-S1PA-matrix.mtx.gz for HTA12_119
Downloaded HT141B1-S1H1-barcodes.tsv.gz for HTA12_126
Downloaded HT141B1-S1H1-features.tsv.gz for HTA12_126
Downloaded HT141B1-S1H1-matrix.mtx.gz for HTA12_126
Downloaded HT163B1-S1H6-barcodes.tsv.gz for HTA12_127
Downloaded HT163B1-S1H6-features.tsv.gz for HTA12_127
Downloaded HT163B1-S1H6-matrix.mtx.gz for HTA12_127
Downloaded HT206B1-S1H4-barcodes.tsv.gz for HTA12_129
Downloaded HT206B1-S1H4-features.tsv.gz for HTA12_129
Downloaded HT206B1-S1H4-matrix.mtx.gz for HTA12_129
Downloaded HT214B1-S1H2-barcodes.tsv.gz for HTA12_130
Downloaded HT214B1-S1H2-features.tsv.gz for HTA12_130
Downloaded HT214B1-S1H2-matrix.mtx.gz for HTA12_130
Downloaded HTAPP-2

In [21]:
import scanpy as sc
import pandas as pd
from pathlib import Path

# Directory where Zhergham and David downloaded the data
data_directory = Path('Data_Davi/')


# Process each folder in the data directory to obtain singld h5ad files.
for folder_path in data_directory.iterdir():
    if folder_path.is_dir():
        # Extract participant ID from folder name
        participant_id = folder_path.name

        # Find files in the folder with the specified extensions
        barcodes_files = list(folder_path.glob('*barcodes.tsv.gz'))
        features_files = list(folder_path.glob('*features.tsv.gz'))
        matrix_files = list(folder_path.glob('*matrix.mtx.gz'))

        # Construct file paths based on found files
        barcodes_path = barcodes_files[0]
        features_path = features_files[0]
        matrix_path = matrix_files[0]
       
        prefix = matrix_path.name.replace("matrix.mtx.gz","")
        print(matrix_path)
        # Load the matrix file with the extracted participant ID
        adata = sc.read_10x_mtx(folder_path,prefix=prefix)

        # Load and add the barcodes and features (genes)
        adata.obs_names = pd.read_csv(barcodes_path, header=None)[0].tolist()
        adata.var_names = pd.read_csv(features_path, header=None)[0].tolist()

        # Save the AnnData object for this participant
        adata.write(f"{folder_path}_adata.h5ad")

        print(f"Processed data for {participant_id}")





Data_Davi/HTA1_878/HTAPP-878-SMP-7149_TST_channel1_matrix.mtx.gz
Processed data for HTA1_878
Data_Davi/HTA12_129/HT206B1-S1H4-matrix.mtx.gz
Processed data for HTA12_129
Data_Davi/HTA1_382/HTAPP-382-SMP-1441_fresh_channel1_matrix.mtx.gz
Processed data for HTA1_382
Data_Davi/HTA1_213/HTAPP-213-SMP-6752_TST_channel1_matrix.mtx.gz
Processed data for HTA1_213
Data_Davi/HTA12_119/HT035B1-S1PA-matrix.mtx.gz
Processed data for HTA12_119
Data_Davi/HTA1_627/HTAPP-627-SMP-3242_fresh_channel1_matrix.mtx.gz
Processed data for HTA1_627
Data_Davi/HTA12_126/HT141B1-S1H1-matrix.mtx.gz
Processed data for HTA12_126
Data_Davi/HTA1_908/HTAPP-908-SMP-4412_fresh_channel1_matrix.mtx.gz
Processed data for HTA1_908
Data_Davi/HTA1_231/HTAPP-231-SMP-6758_TST_channel1_matrix.mtx.gz
Processed data for HTA1_231
Data_Davi/HTA1_983/HTAPP-983-SMP-7659_fresh_channel1_matrix.mtx.gz
Processed data for HTA1_983
Data_Davi/HTA1_963/HTAPP-963-SMP-4741_CST_channel2_matrix.mtx.gz
Processed data for HTA1_963
Data_Davi/HTA1_394/H

In [ ]:
import scanpy as sc
from pathlib import Path

# Directory path where h5ad files are located

directory_path = Path('Data_Davi/')
# List of AnnData objects for each participant 
list_of_adata_objects = []
participant_ids = []

# Iterate over all h5ad files in the directory
for file_path in directory_path.glob("*.h5ad"):
    # Extract participant ID from the file name (assuming a specific naming convention)
    participant_id = file_path.stem  # Adjust this line based on your file naming convention
    print(participant_id)
    # Load the AnnData object
    adata = sc.read_h5ad(file_path)
    # Make observation names unique
    # adata.obs_names_make_unique()
    # Add a new column to .obs with the participant ID
    adata.obs['participant_id'] = participant_id
    print(adata.obs_names)
    # Append to the list
    list_of_adata_objects.append(adata)
    participant_ids.append(participant_id)
print(participant_ids)

In [31]:
participant_ids,list_of_adata_objects

(['HTA1_878_adata',
  'HTA12_129_adata',
  'HTA1_382_adata',
  'HTA1_213_adata',
  'HTA12_119_adata',
  'HTA1_627_adata',
  'HTA12_126_adata',
  'HTA1_908_adata',
  'HTA1_231_adata',
  'HTA1_983_adata',
  'HTA1_963_adata',
  'HTA1_394_adata',
  'HTA1_514_adata',
  'HTA12_127_adata',
  'HTA1_812_adata',
  'HTA12_130_adata',
  'HTA1_997_adata',
  'HTA1_887_adata',
  'HTA1_562_adata',
  'HTA1_525_adata',
  'HTA1_814_adata',
  'HTA12_118_adata'],
 [AnnData object with n_obs × n_vars = 6794880 × 33538
      obs: 'participant_id'
      var: 'gene_ids', 'feature_types',
  AnnData object with n_obs × n_vars = 1756855 × 36601
      obs: 'participant_id'
      var: 'gene_ids', 'feature_types',
  AnnData object with n_obs × n_vars = 737280 × 33538
      obs: 'participant_id'
      var: 'gene_ids', 'feature_types',
  AnnData object with n_obs × n_vars = 6794880 × 33538
      obs: 'participant_id'
      var: 'gene_ids', 'feature_types',
  AnnData object with n_obs × n_vars = 1855232 × 36601
      o

In [ ]:
# Concatenate all AnnData objects into one
combined_adata = sc.concat(list_of_adata_objects, label='participant_id', keys=participant_ids)
# combined_adata = sc.concat(list_of_adata_objects)
# combined_adata.obs['patient_id'].value_counts()

In [ ]:
# !pip install hdf5plugin
import hdf5plugin
combined_adata.write_h5ad(
    "/sbgenomics/output-files/combined_adata.h5ad",
    compression=hdf5plugin.FILTERS["zstd"]
)